# Bioturbation model - calibration using observed data

The bioturbation model relies on an empirical parameter $\beta$, which you may wish to calibrate based on observed data. This notebooks shows how you might go about doing that.

Say you have the data in [example_data.csv](./example_data.csv), which gives observed concentrations at 7, 14, 21 and 28 days in three soil layers (top, middle, bottom):

In [2]:
import pandas as pd

df = pd.read_csv('./example_data.csv')
df

,day,layer,conc
0,0,top,6.625667
1,0,middle,0.000000
2,0,bottom,0.000000
3,7,top,5.184575
4,7,middle,0.569111
5,7,bottom,0.193135
6,14,top,2.737534
7,14,middle,0.657221
8,14,bottom,0.231144
9,21,top,3.259800


*Work in progress...*